In [ ]:
import pandas as pd
import os

In [ ]:
directory = '/Users/mrinaalarora/Downloads/Spotify Extended Streaming History'
dataframes = []

for filename in os.listdir(directory):
    if filename.endswith('.json'):
        filepath = os.path.join(directory, filename)
        df = pd.read_json(filepath)
        dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.to_json('spotify_data.json', orient='records', lines=True)

In [ ]:
my_data = pd.read_json('data/spotify_data.json',lines=True)
my_data

In [ ]:
# Data Preparation and Cleaning
relevant_columns = [ "ts", "platform", "ms_played", "conn_country", "master_metadata_track_name", "master_metadata_album_artist_name", "master_metadata_album_album_name", "shuffle", "skipped", "offline", "incognito_mode" ]
spotify_stream_df = my_data[relevant_columns]
spotify_stream_df # to read all the dataframe at once

In [ ]:
# Number of unique artists
unique_artists = spotify_stream_df['master_metadata_album_artist_name'].nunique()
print(f"Number of unique artists: {unique_artists}")

In [ ]:
# Number of unique tracks
unique_tracks = spotify_stream_df['master_metadata_track_name'].nunique()
print(f"Number of unique tracks: {unique_tracks}")

In [36]:
# Converting 'ts' to datetime
spotify_stream_df['Play-Time'] = pd.to_datetime(spotify_stream_df['ts'])

# Extract year, month, day, weekday, time, hour, day_name
spotify_stream_df['year'] = pd.DatetimeIndex(spotify_stream_df["Play-Time"]).year
spotify_stream_df['month'] = pd.DatetimeIndex(spotify_stream_df["Play-Time"]).month
spotify_stream_df['day'] = pd.DatetimeIndex(spotify_stream_df["Play-Time"]).day
spotify_stream_df['weekday'] = pd.DatetimeIndex(spotify_stream_df["Play-Time"]).weekday
spotify_stream_df['time'] = pd.DatetimeIndex(spotify_stream_df["Play-Time"]).time
spotify_stream_df['hours'] = pd.DatetimeIndex(spotify_stream_df["Play-Time"]).hour
spotify_stream_df['day-name'] = spotify_stream_df["Play-Time"].apply(lambda x: x.day_name())

# Add Count column
spotify_stream_df['Count'] = 1

# Convert milliseconds played to timedeltas
spotify_stream_df["Time-Played (hh-mm-ss)"] = pd.to_timedelta(spotify_stream_df["ms_played"], unit='ms')

# Functions to extract hours and minutes from timedeltas
def hours(td):
    '''To get the hour information'''
    return td.seconds / 3600

def minutes(td):
    '''To get the minutes information'''
    return (td.seconds / 60) % 60

# Apply functions to extract listening time in hours and minutes
spotify_stream_df["Listening Time (Hours)"] = spotify_stream_df["Time-Played (hh-mm-ss)"].apply(hours).round(3)
spotify_stream_df["Listening Time (Minutes)"] = spotify_stream_df["Time-Played (hh-mm-ss)"].apply(minutes).round(3)

,ts,platform,ms_played,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,shuffle,skipped,offline,...,month,day,weekday,time,hours,day-name,Count,Time-Played (hh-mm-ss),Listening Time (Hours),Listening Time (Minutes)
95995,2023-09-13T22:03:08Z,ios,70774,US,Softly,Karan Aujla,Making Memories,True,0.0,0.0,...,9,13,2,22:03:08,22,Wednesday,1,0 days 00:01:10.774000,0.019,1.167
84637,2023-12-20T04:22:05Z,ios,1420,US,Infinite Return,Diljit Dosanjh,Ghost,False,1.0,1.0,...,12,20,2,04:22:05,4,Wednesday,1,0 days 00:00:01.420000,0.000,0.017
29242,2023-03-17T13:48:22Z,ios,354120,US,Dungtitled (In a Major),Stars Of The Lid,And Their Refinement of the Decline,True,0.0,0.0,...,3,17,4,13:48:22,13,Friday,1,0 days 00:05:54.120000,0.098,5.900
37838,2024-03-04T20:04:42Z,ios,210810,US,VULTURES,¥$,VULTURES 1,False,1.0,0.0,...,3,4,0,20:04:42,20,Monday,1,0 days 00:03:30.810000,0.058,3.500
31116,2023-03-26T12:54:41Z,ios,197763,US,Kho Gaye,Taaruk Raina,Mismatched: Season 2 (Soundtrack from the Netf...,True,0.0,0.0,...,3,26,6,12:54:41,12,Sunday,1,0 days 00:03:17.763000,0.055,3.283
